In [1]:
!pip install -Uqqq pip
!pip install langchain
!pip install chromadb
!pip install sentence-transformers
!pip install auto-gptq
!pip install einops
!pip install unstructured
!pip install transformers
!pip install torch
!pip install langchain_community

  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2


In [2]:
!pip install sentence-transformers

In [3]:
from pathlib import Path

In [5]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, GenerationConfig, TextStreamer, pipeline

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [7]:
questions_dir = Path("global-korea-scholarship")
def write_file(question, answer, file_path):
    text = f"""
    Q:{question}
    A:{answer}
    """.strip()
    with Path(questions_dir / file_path).open("w") as text_files:
        text_files.write(text)


In [8]:
write_file(
    question="Should I submit my parents’ passport copy?",
    answer="It is necessary to submit an official document (i.e., applicant’s birth certificate or family register) that proves you and your parents’ relationship. If the document does not include you and your parents’ citizenship information, then you should submit additional documents (i.e., passport copy or other government issued document that indicates citizenship information) for proof of citizenship. Please note that a birth place or ethnic group information on a birth certificate or current residency in the submitted documents will not be accepted as a proof of citizenship.",
    file_path="question_1.txt",
)

In [9]:
write_file(
    question="My father has deceased long time ago, and I cannot provide his proof of citizenship.",
    answer="If you are not able to issue your parent(s)’ proof of citizenship due to your parent(s)’ absence (e.g., divorced, deceased, etc.), then please provide supplementary documents that explains the reason for not submitting their proof of citizenship. (e.g., divorce certificate or death certificate).",
    file_path="question_2.txt",
)

In [10]:
write_file(
    question="My parents and I hold different citizenship. Am I eligible to apply for the program?",
    answer="Yes you may. As long as you and your parent(s) do not hold Korean citizenship, and if you have a citizenship of the country that is invited to the GKS degree program, you are eligible to apply.",
    file_path="question_3.txt",
)

In [11]:
write_file(
    question="My country is not listed in the 2023 GSK-G program. Can I apply for the program?",
    answer="The quota allocation is decided based on mutual agreements on cultural and educational cooperation between Korea and the participating countries. All applicants must hold a citizenship of the GKS partner country except for the R&D program. R&D program is open to all countries around the world. R&D program applicants must directly apply to their designated universities since the program is only available under a university track.",
    file_path="question_4.txt",
)

In [12]:
write_file(
    question="I am expected to graduate from my degree program in March 2024. Am I eligible to apply for this program?",
    answer="Yes. Applicants who are expected to graduate by July 31st, 2024 can also apply for the program with a certificate of expected graduation. Such applicants must submit their official graduation certificate (or diploma) and final academic transcript to GKS Center, NIIED by July 31st, 2024. Failure to do so will result in the cancellation of your acceptance.",
    file_path="question_5.txt",
)

In [13]:
write_file(
    question="I have a master’s degree. Can I apply for a master’s degree program in a different major? Which degree certificate should I submit?",
    answer="You can apply for a master’s program while holding another master’s degree. If you are applying to a master’s degree program, you should submit a degree certificate and academic transcript of your bachelor’s degree. In this case, your CGPA of your bachelor’s degree should meet the grade requirement in the application guidelines. Providing your master’s degree certificate and academic transcript is optional.",
    file_path="question_6.txt",
)

In [14]:
write_file(
    question="I am currently in my master’s degree program at a university in Korea. Am I eligible to apply for the master’s degree program at another university?",
    answer="If you have graduated from a Korean high school or a Korean university, or if are currently enrolled in the final year of your degree curriculum (or if you are about to obtain a degree) at a Korean University, you cannot apply for the GKS degree program. If you are not near the graduation, you may apply for the program. However, if you are selected as our GKS scholar, you must drop out of the Korean university you are currently enrolled at and start the degree program under GKS as a first year. Transfer credits from your previous university will not be accepted. Applicants who are residing in Korea may need to return to their own country in order to issue a new visa for their Korean language program/degree program. In this case, their entry airfare will not be provided.",
    file_path="question_7.txt",
)

In [15]:
write_file(
    question="The grades in my transcript are different from the GPA scales in the guidelines. How should I submit my CGPA? Can I use the scores I converted from a grade conversion website?",
    answer="If your grades do not belong to any of the GPA scales we accept (4.0, 4.3, 4.5, 5.0 or 100 point scale), you must submit a converted grade confirmed by the university. If applicants convert their grades through a grade conversion tool such as scholaro.com, the converted CGPA must be officially acknowledged by the relevant university. Please submit an official certificate or at least a testimonial issued by your university that confirms the converted CGPA. This supplementary document does not need to be apostilled or consular confirmed but must be submitted in an original document with the university’s authentication. The screening committee will determine whether your CGPA is eligible for the program or not based on the information you provide.",
    file_path="question_8.txt",
)

In [16]:
write_file(
    question="My CGPA is below 80%. Am I not eligible to apply for the program?",
    answer="You are eligible to apply if you are ranked within the top 20% of your class even if your score percentile is below 80%. Along with your original transcript, please submit an official certificate or a letter from your university that your grade stands within the top 20% of your class.",
    file_path="question_9.txt",
)

In [17]:
write_file(
    question="My academic transcript only indicates CGPA and does not offer GPA by semester. What should I put in the application form?",
    answer="If your transcript does not provide GPA of each semester/year, you do not need to fill in the box for each semester/year. However, you must put your CGPA AND a score percentile of the entire degree curriculum. If your transcript does not provide a score percentile, refer to Appendix “GPA Conversion Table” to convert your CGPA into a 100 point scale.",
    file_path="question_10.txt",
)

In [18]:
write_file(
    question="I am applying for a doctoral degree program. Should I also submit a CGPA of my bachelor’s degree?",
    answer="Applicants for a doctoral degree program should put a CGPA of their previous degree (master’s degree) on the application form. Doctoral degree program applicants should submit the graduation certificate and official academic transcript of both bachelor’s and master’s degree. The CGPA required in the application is of the master’s degree.",
    file_path="question_11.txt",
)

In [19]:
write_file(
    question="I transferred to my university, but the official academic transcript issued by my university does not indicate the GPA of my previous university. How should I submit my academic record?",
    answer="Please submit an official academic transcript issued by the university you graduated from. If the transcript does not indicate your grades of the entire degree curriculum, please submit your academic records of the previous university as a supplementary document as a reference material. The CGPA required in the application should be the one shown on the official academic transcript from the university you have graduated from.",
    file_path="question_12.txt",
)

In [20]:
write_file(
    question="I meet all the application requirements listed in the application guideline. Does this mean that I am qualified to apply to all GKS partner universities?",
    answer="Not necessarily. The application requirements we have indicated in the application guideline are the minimum requirements to apply for the GKS program. Some universities and departments may have additional requirements such as certain level of language proficiency. Therefore, we ask all prospective applicants to check the “University information” files posted with the application guideline to find required qualifications set by each university/department. If a university requests additional material (e.g., portfolio) beyond NIIED’s required documents, you should submit those materials directly to the university.",
    file_path="question_13.txt",
)

In [21]:
write_file(
    question="I am currently living in Korea. Can I apply for this program?",
    answer="Yes, you may apply for the program while residing in a different country. Applicants must apply either through a Korean embassy where they hold citizenship (embassy track) or through a Korean university (university track). Embassy track applicants who are applying from a third country must check the method of application submission or interview process with the embassy. Successful candidates who are currently residing in Korea or enter Korea from a third country will not be provided with the entry airfare. Please be noted that it may take more time and effort to obtain necessary authentications (apostille, consular confirmation, certified translation, etc.) on required documents from abroad.",
    file_path="question_14.txt",
)

In [22]:
write_file(
    question="May I apply for the program by submitting my application to NIIED directly?",
    answer="No, you may not. Applicants must submit their application documents to the GKS selection department at a Korean embassy or a university they are applying to. NIIED only accepts applications from embassies or universities through official documents and does not accept applications from individual applicants. Any documents mistakenly sent to NIIED will be excluded from the screening and will not be returned.",
    file_path="question_15.txt",
)

In [23]:
write_file(
    question="When is the deadline for the application?",
    answer="Each embassy/university has different application period. Please check the website of each institution to find out the application deadline. Embassies and universities that conduct the first round of selection have different application submission guidelines, so please check with the embassy/university regarding any submission related inquiries such as application deadline, submission method, etc.",
    file_path="question_16.txt",
)

In [24]:
write_file(
    question="I am a citizen of China/Czech Republic. In the application guidelines, there is quota only assigned in the embassy track for the Chinese/Czech citizens. Does that mean that I must apply to the Korean Embassy in my country?",
    answer="If you are a Chinese citizen, you should submit your applications to China Scholarship Council (CSC) under the Ministry of Education in China instead of the Korean Embassy in China. This is based on the mutual agreement between Korean and China. Likewise, a Czech citizen must also submit the application to the Ministry of Education in Czech Republic, in accordance with the mutual agreement between the governments of Korea and Czech Republic.",
    file_path="question_17.txt",
)

In [25]:
write_file(
    question="I am a U.S. citizen living in Los Angeles. I am going to apply through an embassy track. Can I submit my application to the Korean Consulate in L.A. instead of the Korean Embassy in Washington D.C.?",
    answer="Applicants from the United States of America and Vietnam can submit their application to the nearby Korean consulate. Korean embassy in Washington and Korean consulate in L.A. will cooperate to conduct the first round of selection. Please contact the Korean embassy for further information. Embassy track applicants in Vietnam can also submit their application documents to embassy/consulate in Hanoi, Danang, or Ho Chi Minh.",
    file_path="question_18.txt",
)

In [26]:
write_file(
    question="For the legalization on “Required certificates”, can I choose between apostille or consular confirmation?",
    answer="Countries that have joined the Apostille Convention, can only issue an apostille. For the rest of the countries, you should obtain consular confirmation from the Korean embassy/consulate in your country. If your country does not belong to the Apostille Convention or if the document cannot be apostilled, please obtain a consular confirmation from the Korean embassy /consulate.",
    file_path="question_19.txt",
)

In [27]:
write_file(
    question="Can I submit an original degree certificate and academic transcript without apostille/consular confirmation?",
    answer="The “Required certificates” must be submitted with an apostille or consular confirmation. Original documents without additional authentication, notarized documents certified by only a notary public, or certified documents confirmed by the other government agencies will not be accepted. (Exception: certificate issued by Korean government or institutions should be submitted in their original version without additional authentication.)",
    file_path="question_20.txt",
)

In [28]:
write_file(
    question="[Embassy track applicant] Should I apostille my birth certificate four times?",
    answer="Embassy track applicants should submit ONE set of original application documents (that includes apostilled/consular-confirmed documents) and THREE sets of photocopied application documents. Therefore, embassy track applicants do not need to have their certificates authenticated four times.",
    file_path="question_21.txt",
)

In [29]:
write_file(
    question="I obtained an apostille on my graduation certificate in the past. Can I submit a copy of this document?",
    answer="It will not be accepted as the document will be considered as a simple photocopy. It does not matter whether it is photocopied in color or black and white. Your document will be accepted only if the government agency that initially issued the apostille issues a certified true copy or if you obtain a certified true copy (등본인증) of your document from a Korean embassy.",
    file_path="question_22.txt",
)

In [30]:
write_file(
    question="How should I submit the documents with translation?",
    answer="All application documents that are not written in Korean or English must be accompanied with a certified translation. For “Required certificates”, apostille or consular confirmation is only required either on original documents or on certified translations. If the document is written in both languages (original language and English), it does not need to be translated again.",
    file_path="question_23.txt",
)

In [31]:
write_file(
    question="I will submit my original graduation certificate with an apostille but it cannot be reissued any more. Can it be returned?",
    answer="All application materials submitted to NIIED WILL NOT BE RETURNED IN ANY CASE. Please obtain an apostille or consular confirmation on a certified copy. If you do not pass the first round of selection, the embassy/university may (or may not) return your documents based on their policy. Please note that the graduation certificate submitted at the time of application will only be used for the selection procedure. Therefore, selected GKS scholars should prepare additional graduation certificates with an apostille/consular confirmation so they can submit it to different institutions to apply for a visa (embassy), to apply for a standard admission letter (university), or to change visa type (Korean immigration office) in the future.",
    file_path="question_24.txt",
)

In [32]:
write_file(
    question="I am curious about the validity of the authenticated documents. Can I submit my graduation certificate that was apostilled last year?",
    answer="If there is an expiration date on an apostilled or consular confirmed document, the validity of the document will be judged based on the date indicated on the authentication. If there is no specific date indicated, the document will be accepted if it was submitted within 2 years from the date of authentication.",
    file_path="question_25.txt",
)

In [33]:
write_file(
    question="I have graduated from another country. Therefore, I cannot obtain an apostille on my certificates from my government or apply for a consular confirmation from the Korean embassy in my country. How should I submit my graduation certificate and academic transcript that were issued from another country?",
    answer="If your documents were issued from the university or from a government abroad, you should obtain an apostille if the relevant country is a member of the Apostille Convention. If you are not able to obtain an apostille from abroad, please check if you can obtain a consular confirmation from the embassy/consulate of the relevant country.",
    file_path="question_26.txt",
)

In [34]:
write_file(
    question="How many departments and universities can I apply for admission?",
    answer="If you apply through a university track, you may choose only ONE university and ONE department. If you apply through an embassy track, you may choose up to THREE universities under Type A and B. Please remember that at least one of your choices must be from Type B universities. Please note that you may choose the same department for all three universities, but cannot choose three different departments from a single university.",
    file_path="question_27.txt",
)

In [35]:
write_file(
    question="Are there tips on choosing three universities?",
    answer="Embassy track applicants should understand that passing the second round of selection will not guarantee an admission to the university that you have chosen. Embassy track applicants who only choose highly competitive universities can fail from all three universities in the third round of selection. Each Korean university has recognition for variety of fields and has its own strength. Please conduct your own research to find out which university is the leading institution in the field of study you would like to pursue. Please also consider the fact that the cost of living for students at universities in the metropolitan area is higher than those in non-metropolitan universities. For your reference, GKS scholars pay for their accommodation within the stipend they receive each month.",
    file_path="question_28.txt",
)

In [36]:
write_file(
    question="Who should I obtain my recommendation letter from?",
    answer="We recommend you receive your recommendation letter from someone who can provide thorough assessment of your academic abilities such as a professor or a dean of the department at your previous university. The recommendation letter should be dated within a year from the last date of the application deadline. Applicants who prepared their recommendation letter in advance may use the previous year’s (2022) recommendation letter form. Only one recommendation letter is required, but applicants may submit up to two recommendation letters if necessary.",
    file_path="question_29.txt",
)

In [37]:
write_file(
    question="Must I submit the certificates of language proficiency (English or Korean)?",
    answer="The certificate of English proficiency (TOEIC, TOEFL, IELTS) or Korean proficiency (TOPIK) is optional, which means you may apply for GKS without it. However, some universities or departments may request applicants with certain level of language proficiency. It is helpful to provide supporting documentation that proves your abilities, including language skills. A proof that English was used as a medium of instruction cannot be a substitute for an English proficiency test certificate.",
    file_path="question_30.txt",
)

In [38]:
write_file(
    question="Should I include a detailed research proposal in the Study Plan?",
    answer="Yes. You must at least describe the general direction and outlines of your study plans, if not in full detail. Please elaborate on your study objectives in Korea, tangible plans during and after you complete the degree program.",
    file_path="question_31.txt",
)

In [39]:
write_file(
    question="I am planning to apply through a university track. Can I submit my application to multiple universities?",
    answer="University track applicants can only apply to one university and one department. If an applicant is found to have applied to more than two universities, his/her application document will be excluded from the next round of evaluation. Even after the announcement of qualified applicants, scholarships may be cancelled. Embassy track applicants whose application have not passed the first round may apply to a university track once more. However, applicants cannot apply for both tracks at once.",
    file_path="question_32.txt",
)

In [40]:
write_file(
    question="My name on my birth certificate is different from the one on my graduation certificate. What should I do when the names are different on the documents?",
    answer="Applicant must apply with the name on their passport. If the name on your passport is different from other documents, please provide a supplementary document issued by your government that explains the discrepancy.",
    file_path="question_33.txt",
)

In [41]:
write_file(
    question="How will I know if I have passed each assessment?",
    answer="Result of the first round of selection will be announced by each embassy and university. Result of the second round and the list of final successful candidates will be announced on the GKS Notice board at studyinkorea.go.kr website. Important notices for all applicants will be released on the GKS Notice board, so all applicants must check the website during the selection period.",
    file_path="question_34.txt",
)

In [42]:
write_file(
    question="Is it mandatory for all GKS scholars to pass at least Level 3 in TOPIK after one year of Korean Language Program?",
    answer="Yes. Every GKS scholar (except scholars in a research program) must achieve at least Level 3 on TOPIK (Test of Proficiency in Korean) within a yearlong Korean language program to proceed to his or her degree program. In addition, certain university or department requires students to submit a valid TOPIK certificate of level 4 or higher. Scholars who applied to such department may not enter their degree program if they fail to achieve the required TOPIK certificate during a yearlong Korean language program.",
    file_path="question_35.txt",
)

In [43]:
write_file(
    question="If I become a GKS scholar, will I be assigned to a Korean language institution at a different region of my degree university?",
    answer="No. From 2023, all GKS scholars will take their Korean language program at the same university where they will conduct their degree program. Exception: KAIST, UNIST, GIST, KDI School of Public Policy and Management, POSTECH will designate another university for a Korean language program. During the language program, scholars are required to live on-campus. In general, scholars are assigned to a dormitory with roommate(s). Each institution has different facilities. If an independent kitchen or a bathroom is mandatory for religious reasons, etc., please check with the university of your interest.",
    file_path="question_36.txt",
)

In [44]:
write_file(
    question="The degree program I applied to is taught in English only. May I be exempt from the Korean Language Program?",
    answer="A yearlong Korean language program is an important part of the GKS program. All GKS scholars must take a Korean language program regardless of the curriculum of the degree program. However, if you have already attained advanced level (Level 5 or 6) on TOPIK when applying for the program, you must submit your valid TOPIK certificate to be exempt from Korean language program. You will directly start your degree program from September 2023. Applicants who attained TOPIK level 5 or 6 after the initial application deadline must submit the valid TOPIK certificate to NIIED by June 14, 2023. (*Acceptable TOPIK certificates: The 74 st ~ 86 th score). If you attain TOPIK level 5 or 6 during the first 6 months of Korean language program, you will be exempt from the remaining 6-month language program, and you must start your degree program in the following semester (March, 2024).",
    file_path="question_37.txt",
)

In [45]:
write_file(
    question="May I change my final university choice during my Korean language program or during a degree program?",
    answer="No, you cannot. Changing the university or transferring to another university is NOT PERMITTED under any circumstances.",
    file_path="question_38.txt",
)

In [46]:
write_file(
    question="When can I apply for Korean Government Scholarship Program?",
    answer="For undergraduate program, the announcement will be posted on the Korean Embassy website around August; and for graduate program, the announcement will be made around February.",
    file_path="question_39.txt",
)

In [47]:
write_file(
    question="Are the courses in Korean university conducted in Korean or English?",
    answer="Nowadays English courses are offered by many universities in Korea. You need to check carefully before you select the universities by visiting the English website of the university you are interested in.",
    file_path="question_40.txt",
)

In [48]:
write_file(
    question=" I cannot speak Korean at all; can I still apply for this program?",
    answer="Yes, you can. Scholarship grantees are required to take a preliminary Korean language course at the institution designated by NIIED for 1 year. However, the grantees with level 5 or higher in TOPIK (Test of Proficiency in Korean) can be exempted from the preliminary Korean language course.",
    file_path="question_41.txt",
)

In [49]:
model_name_or_path = "TheBloke/Nous-Hermes-13B-GPTQ"
#model_basename_q = "nous-hermes-13b-GPTQ-4bit-128g.no-act.order"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    use_safetensors=True,
    trust_remote_code=True,
    device=device
)

generation_config = GenerationConfig.from_pretrained(model_name_or_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
INFO - The layer lm_head is not quantized.
INFO:auto_gptq.modeling._base:The layer lm_head is not quantized.


In [50]:
question = (
    "What Programming language should I learn as a beginner?"
)

prompt = f"""
### instructions: {question}
### Response:
""".strip()

print(prompt)

### instructions: What Programming language should I learn as a beginner?
### Response:


In [51]:
%%time
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

with torch.inference_mode():
  output = model.generate(inputs = input_ids, temperature=0.7, max_new_tokens=512)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


CPU times: user 2.46 s, sys: 168 ms, total: 2.63 s
Wall time: 3.29 s


In [52]:
print(tokenizer.decode(output[0]))


<s> ### instructions: What Programming language should I learn as a beginner?
### Response:As a beginner, I would recommend learning Python. It is a versatile and easy-to-understand programming language with a large community of support and a wide range of applications.</s>


In [53]:
generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "pad_token_id": 0
}

In [54]:
streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False
)


In [55]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    generation_config=generation_config,
    streamer=streamer,
    batch_size=1
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCa

In [56]:
llm = HuggingFacePipeline(pipeline=pipe)

In [57]:
response = llm(prompt)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


As a beginner, you can start with Python. It is an easy-to-learn programming language that has a simple syntax and a large community for support.


**EMBEDDING STEP**

In [58]:
embeddings = HuggingFaceEmbeddings(
    model_name="embaas/sentence-transformers-multilingual-e5-base",
    model_kwargs={"device": device},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.79k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [59]:
loader = DirectoryLoader("./global-korea-scholarship/", glob="**/*txt")
documents = loader.load()
len(documents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


41

In [60]:
text_splitter = CharacterTextSplitter(chunk_size = 512, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [61]:
print(texts[0])
print(texts[2])
print(texts[4])

page_content='Q:I transferred to my university\nbut the official academic transcript issued by my university does not indicate the GPA of my previous university. How should I submit my academic record?' metadata={'source': 'global-korea-scholarship/question_12.txt'}
page_content='Q:I am applying for a doctoral degree program. Should I also submit a CGPA of my bachelor’s degree?\n\nA:Applicants for a doctoral degree program should put a CGPA of their previous degree (master’s degree) on the application form. Doctoral degree program applicants should submit the graduation certificate and official academic transcript of both bachelor’s and master’s degree. The CGPA required in the application is of the master’s degree.' metadata={'source': 'global-korea-scholarship/question_11.txt'}
page_content='Q:If I become a GKS scholar, will I be assigned to a Korean language institution at a different region of my degree university?' metadata={'source': 'global-korea-scholarship/question_36.txt'}


In [62]:
db = Chroma.from_documents(texts, embeddings)
db.similarity_search("GPA")

[Document(page_content='Q:The grades in my transcript are different from the GPA scales in the guidelines. How should I submit my CGPA? Can I use the scores I converted from a grade conversion website?', metadata={'source': 'global-korea-scholarship/question_8.txt'}),
 Document(page_content='Q:I transferred to my university\nbut the official academic transcript issued by my university does not indicate the GPA of my previous university. How should I submit my academic record?', metadata={'source': 'global-korea-scholarship/question_12.txt'}),
 Document(page_content='Q:My academic transcript only indicates CGPA and does not offer GPA by semester. What should I put in the application form?\n\nA:If your transcript does not provide GPA of each semester/year, you do not need to fill in the box for each semester/year. However, you must put your CGPA AND a score percentile of the entire degree curriculum. If your transcript does not provide a score percentile, refer to Appendix “GPA Conversio

CONVERSATIONAL CHAIN

In [63]:
template = """
### Instructions: You are a scholarship support agent that is talking to a prospective scholarship applicant. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. if you don't know the answer - say that you don't know.
Keep your replies concise, informative and compassionate
{chat_history}
### Input: {question}
### Response:
""".strip()

In [64]:
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"], template=template
)

In [65]:
memory = ConversationBufferMemory(
    memory_key = "chat_history",
    human_prefix = "### Input",
    ai_prefix = "### Response",
    output_key = "answer",
    return_messages=True
)

In [66]:
chain = ConversationalRetrievalChain.from_llm(
    llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=True
)

In [67]:
question = "I want to apply to GKS, what should I do?"
answer = chain(question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instructions: You are a scholarship support agent that is talking to a prospective scholarship applicant. Use only the chat history and the following information
Q:My country is not listed in the 2023 GSK-G program. Can I apply for the program?

Q:I meet all the application requirements listed in the application guideline. Does this mean that I am qualified to apply to all GKS partner universities?

Q:If I become a GKS scholar, will I be assigned to a Korean language institution at a different region of my degree university?

A:Not necessarily. The application requirements we have indicated in the application guideline are the minimum requirements to apply for the GKS program. Some universities and departments may have additional requirements such as certain level of language proficiency. Therefore, we ask all prospective applicants to check the “University information” files p

In [68]:
question = "when will the result be announced?"
answer = chain(question)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: I want to apply to GKS, what should I do?
Assistant: To apply for the GKS program, please follow these steps:
1. Check the eligibility criteria mentioned in the application guidelines and ensure that you meet them.
2. Research about the participating universities and their programs to see which one suits your academic interests and career goals.
3. Prepare all necessary documents according to the application guidelines, including your transcripts, recommendation letters, essays, etc.
4. Submit your online application through the NIIED website before the deadline.
5. Wait for the results and keep an eye on the NIIED website or your email for any updates regarding the selection process.
Follow Up Input: when will the result be announced?
Standalone question:


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


When is the result announcement date?

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instructions: You are a scholarship support agent that is talking to a prospective scholarship applicant. Use only the chat history and the following information
Q:How will I know if I have passed each assessment?

A:Result of the first round of selection will be announced by each embassy and university. Result of the second round and the list of final successful candidates will be announced on the GKS Notice board at studyinkorea.go.kr website. Important notices for all applicants will be released on the GKS Notice board, so all applicants must check the website during the selection period.

Q:Who should I obtain my recommendation letter from?

Q:Must I submit the certificates of language proficiency (English or Korean)?

Q:Are there tips on choosing three universities?
to answer in a helpful manner to the question. if y

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The result announcement dates vary depending on the specific program and country. Please refer to the official website or contact the relevant embassy for more details.

> Finished chain.

> Finished chain.


FORMATTING

In [69]:
memory = ConversationBufferMemory(
    memory_key = "chat_history",
    human_prefix = "### Input",
    ai_prefix = "### Response",
    input_key = "question",
    output_key = "output_text",
    return_messages=False
)

chain = load_qa_chain(
    llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True
)

In [70]:
question = "when is the result of GKS is announced?"
docs = db.similarity_search(question)
answer = chain.run({"input_documents":docs, "question":question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instructions: You are a scholarship support agent that is talking to a prospective scholarship applicant. Use only the chat history and the following information
Q:How will I know if I have passed each assessment?

A:Result of the first round of selection will be announced by each embassy and university. Result of the second round and the list of final successful candidates will be announced on the GKS Notice board at studyinkorea.go.kr website. Important notices for all applicants will be released on the GKS Notice board, so all applicants must check the website during the selection period.

Q:My country is not listed in the 2023 GSK-G program. Can I apply for the program?

Q:I am expected to graduate from my degree program in March 2024. Am I eligible to apply for this program?

A:Yes. Applicants who are expected to graduate by July 31st, 2024 can also apply for the program w

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


The result of GKS varies depending on the program and the country. Generally, the first round of selection results are announced by each embassy and university in December or January. The second round and the list of final successful candidates are usually announced in April or May through the GKS notice board at studyinkorea.go.kr website. It is recommended to regularly check the website for updates and important notices throughout the application process.

> Finished chain.

> Finished chain.


In [71]:
print(answer.strip())

The result of GKS varies depending on the program and the country. Generally, the first round of selection results are announced by each embassy and university in December or January. The second round and the list of final successful candidates are usually announced in April or May through the GKS notice board at studyinkorea.go.kr website. It is recommended to regularly check the website for updates and important notices throughout the application process.


In [77]:
DEFAULT_TEMPLATE = """
### Instructions: You are a scholarship support agent that is talking to a prospective scholarship applicant. Use only the chat history and the following information
{context}
to answer in a helpful manner to the question. if you don't know the answer - say that you don't know.
Keep your replies concise, informative and compassionate
{chat_history}
### Input: {question}
### Response:
""".strip()

class Chatbot:
  def __init__(
      self,
      text_pipeline: HuggingFacePipeline,
      embeddings: HuggingFaceEmbeddings,
      documents_dir: Path,
      prompt_template: str = DEFAULT_TEMPLATE,
      verbose: bool = False):
      prompt = PromptTemplate(
          input_variables=["context", "question", "chat_history"], template=template
      )
      self.chain = self._create_chain(text_pipeline, prompt, verbose)
      self.db = self._embed_data(documents_dir, embeddings)

  def _create_chain(
      self,
      text_pipeline: HuggingFacePipeline,
      prompt: PromptTemplate,
      verbose: bool = False,
    ):
      memory = ConversationBufferMemory(
        memory_key = "chat_history",
        human_prefix = "### Input",
        ai_prefix = "### Response",
        input_key = "question",
        output_key = "output_text",
        return_messages=False
      )
      return load_qa_chain(
          text_pipeline,
          chain_type= "stuff",
          prompt=prompt,
          memory=memory,
          verbose=verbose
      )
  def _embed_data(
      self, documents_dir: Path, embeddings: HuggingFaceEmbeddings
  ) -> Chroma:
      loader = DirectoryLoader("./global-korea-scholarship/", glob="**/*txt")
      documents = loader.load()
      text_splitter = CharacterTextSplitter(chunk_size = 512, chunk_overlap=0)
      texts = text_splitter.split_documents(documents)
      return Chroma.from_documents(texts, embeddings)
  def __call__(self, user_input:str) -> str:
    docs = self.db.similarity_search(user_input)
    return self.chain.run({"input_documents":docs, "question":user_input})

In [78]:
chatbot = Chatbot(llm, embeddings, "./global-korea-scholarship/")

In [79]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

while True:
  user_input = input("You: ")
  if user_input.lower() in ['done', 'ok']:
      break
  answer = chatbot(user_input)
  print()

You: How to apply to GKS?
To apply for the Global Korea Scholarship (GKS) program, please follow these steps:
1. Check the eligibility criteria on the official website of the GKS program to confirm whether you are eligible or not.
2. If you meet the eligibility criteria, read the application guidelines carefully and make sure you understand them before starting the application process.
3. Fill out the online application form completely and accurately, making sure to attach all required documents such as transcripts, diplomas, and language test scores.
4. Submit your application by the deadline specified on the GKS website.
5. Wait for the results of the selection process, which will be announced on the GKS website.
If you have any further questions about the application process, feel free to ask me.

You: What is the required documents?
To apply for the Global Korea Scholarship (GKS), you need to provide the following required documents:
1. A completed online application form
2. An off

In [85]:
question = "How will I know if I have passed each assessment?"
docs = db.similarity_search(question)
answer = chain.run({"input_documents":docs, "question":question})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
### Instructions: You are a scholarship support agent that is talking to a prospective scholarship applicant. Use only the chat history and the following information
Q:I am planning to apply through a university track. Can I submit my application to multiple universities?

Q:I am planning to apply through a university track. Can I submit my application to multiple universities?

Q:I transferred to my university
but the official academic transcript issued by my university does not indicate the GPA of my previous university. How should I submit my academic record?

Q:I transferred to my university
but the official academic transcript issued by my university does not indicate the GPA of my previous university. How should I submit my academic record?
to answer in a helpful manner to the question. if you don't know the answer - say that you don't know.
Keep your replies concise, informa